# Paso 0: Configuración y Carga
Primero, montamos Drive y cargamos ambos datasets.

In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [15]:
# --- 1. Montar Drive y Definir Rutas ---
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [16]:

BASE_DIR = "/content/drive/My Drive/AI Projects/Customer Churn/"
DATA_DIR = BASE_DIR + "datasets/"

# --- 2. Cargar el DataFrame de Clientes (del Notebook 02) ---
# Esta es nuestra tabla BASE
try:
    # (Primero, guarda df_clientes en el Notebook 02)

    df_clientes = pd.read_parquet(DATA_DIR + "df_clientes_rfm.parquet")
    print("✅ DataFrame de Clientes (RFM) cargado.")
    display(df_clientes.head())

except FileNotFoundError:
    print("❌ Error: No se encontró 'df_clientes_rfm.parquet'.")
    print("Por favor, guarda 'df_clientes' en el Notebook 02 antes de continuar.")


# --- 3. Cargar el DataFrame Transaccional (del Notebook 01) ---
# Usaremos este para CALCULAR las nuevas features
try:
    df_transacciones = pd.read_parquet(DATA_DIR + "df_master_transaccional.parquet")

    # Re-filtramos por 'delivered' y convertimos fechas (como en el Notebook 02)
    df_delivered = df_transacciones[df_transacciones['order_status'] == 'delivered'].copy()
    df_delivered['order_purchase_timestamp'] = pd.to_datetime(df_delivered['order_purchase_timestamp'])

    print("\n✅ DataFrame Transaccional (limpio) cargado.")
    display(df_delivered.head())

except FileNotFoundError:
     print("❌ Error: No se encontró 'df_master_transaccional.parquet'.")

# --- 4. Definir la Snapshot Date (DEBE SER LA MISMA que en el Notebook 02) ---
snapshot_date = df_delivered['order_purchase_timestamp'].max() + pd.Timedelta(days=1)
print(f"\nSnapshot Date (consistente): {snapshot_date}")

✅ DataFrame de Clientes (RFM) cargado.


,customer_unique_id,last_purchase_date,frequency,monetary,recency,churn
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90,112,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19,115,0
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22,537,1
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62,321,1
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89,288,1



✅ DataFrame Transaccional (limpio) cargado.


,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,total_payment_value,num_items,total_product_value,total_freight_value,product_id,product_category_name_english
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,38.71,1.0,29.99,8.72,87285b34884572647811a353c7ac498a,housewares
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,141.46,1.0,118.70,22.76,595fac2a385ac33a80bd5114aec74eb8,perfumery
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,179.12,1.0,159.90,19.22,aa4383b373c6aca5d8797843e5594415,auto
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,72.20,1.0,45.00,27.20,d0b61bfb1de832b15ba9d266ca96e5b0,pet_shop
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,28.62,1.0,19.90,8.72,65266b2da20d04dbe00c5c2d3bb7859e,stationery



Snapshot Date (consistente): 2018-08-30 15:00:37


# Paso 1: Feature "Tenure" (Antigüedad del Cliente)
Esta feature es muy predictiva. ¿Es un cliente nuevo o uno antiguo?

Lógica: Tenure = (Fecha de "Hoy") - (Fecha de la *primera* compra)

In [17]:
# --- 1. Calcular Fecha de Primera Compra ---
df_primera_compra = df_delivered.groupby('customer_unique_id')['order_purchase_timestamp'].min().reset_index()
df_primera_compra = df_primera_compra.rename(columns={'order_purchase_timestamp': 'first_purchase_date'})

# --- 2. Unir a nuestro df_clientes ---
df_clientes = pd.merge(df_clientes, df_primera_compra, on='customer_unique_id', how='left')

# --- 3. Calcular Tenure en días ---
df_clientes['tenure_days'] = (snapshot_date - df_clientes['first_purchase_date']).dt.days

print("--- df_clientes con Tenure ---")
display(df_clientes.head())

--- df_clientes con Tenure ---


,customer_unique_id,last_purchase_date,frequency,monetary,recency,churn,first_purchase_date,tenure_days
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90,112,0,2018-05-10 10:56:27,112
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19,115,0,2018-05-07 11:11:27,115
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22,537,1,2017-03-10 21:05:03,537
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62,321,1,2017-10-12 20:29:41,321
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89,288,1,2017-11-14 19:45:42,288


# Paso 2: Features de Promedio de Pedido
¿Es un cliente que gasta mucho en cada pedido, o gasta poco pero con frecuencia?

Lógica: Agrupamos las transacciones por cliente y calculamos la media() (promedio).

In [18]:
# --- 1. Agrupar transacciones y calcular promedios ---
df_promedios = df_delivered.groupby('customer_unique_id').agg(

    # Valor promedio de pago por pedido
    avg_payment_value=('total_payment_value', 'mean'),

    # Número promedio de items por pedido
    avg_items_per_order=('num_items', 'mean'),

    # Costo promedio de envío por pedido
    avg_freight_value=('total_freight_value', 'mean')

).reset_index()

# --- 2. Unir a nuestro df_clientes ---
df_clientes = pd.merge(df_clientes, df_promedios, on='customer_unique_id', how='left')

print("--- df_clientes con Promedios ---")
display(df_clientes.head())

--- df_clientes con Promedios ---


,customer_unique_id,last_purchase_date,frequency,monetary,recency,churn,first_purchase_date,tenure_days,avg_payment_value,avg_items_per_order,avg_freight_value
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90,112,0,2018-05-10 10:56:27,112,141.90,1.0,12.00
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19,115,0,2018-05-07 11:11:27,115,27.19,1.0,8.29
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22,537,1,2017-03-10 21:05:03,537,86.22,1.0,17.22
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62,321,1,2017-10-12 20:29:41,321,43.62,1.0,17.63
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89,288,1,2017-11-14 19:45:42,288,196.89,1.0,16.89


# Paso 3: Features de Categoría de Producto
¿Qué tipo de productos compra este cliente?

Lógica: 1. ¿Cuántas categorías únicas ha comprado? (Diversidad) 2. ¿Cuál es su categoría favorita? (Comportamiento)

In [19]:
# --- 1. Agrupar transacciones y calcular features de categoría ---
df_categorias = df_delivered.groupby('customer_unique_id')['product_category_name_english'].agg(

    # 1. Contar el número de categorías únicas
    n_unique_categories='nunique',

    # 2. Encontrar la categoría más frecuente (la moda)
    # (Usamos una lambda para manejar el caso de que la moda esté vacía)
    main_category=lambda x: x.mode().iloc[0] if not x.mode().empty else None

).reset_index()

# --- 2. Unir a nuestro df_clientes ---
df_clientes = pd.merge(df_clientes, df_categorias, on='customer_unique_id', how='left')

print("--- df_clientes con Categorías ---")
display(df_clientes.head())

--- df_clientes con Categorías ---


,customer_unique_id,last_purchase_date,frequency,monetary,recency,churn,first_purchase_date,tenure_days,avg_payment_value,avg_items_per_order,avg_freight_value,n_unique_categories,main_category
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10 10:56:27,1,141.90,112,0,2018-05-10 10:56:27,112,141.90,1.0,12.00,1,bed_bath_table
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07 11:11:27,1,27.19,115,0,2018-05-07 11:11:27,115,27.19,1.0,8.29,1,health_beauty
2,0000f46a3911fa3c0805444483337064,2017-03-10 21:05:03,1,86.22,537,1,2017-03-10 21:05:03,537,86.22,1.0,17.22,1,stationery
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12 20:29:41,1,43.62,321,1,2017-10-12 20:29:41,321,43.62,1.0,17.63,1,telephony
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14 19:45:42,1,196.89,288,1,2017-11-14 19:45:42,288,196.89,1.0,16.89,1,telephony


# Paso 4: Limpieza Final y Guardado
Nuestro df_clientes ahora es un dataset de "features". Vamos a limpiarlo, eliminar las columnas que no necesitamos para el modelo y guardarlo.

In [20]:
# --- 1. Revisar NaNs (Valores Nulos) ---
print("\nValores Nulos (NaN) antes de la limpieza:")
print(df_clientes.isnull().sum())

# 'main_category' puede tener nulos si un producto no tenía categoría.
# Los llenamos con una etiqueta "unknown".
df_clientes['main_category'] = df_clientes['main_category'].fillna('unknown')


# --- 2. Seleccionar Columnas para el Modelo ---
# Eliminamos columnas de ID, fechas y texto que no sean features
features_finales = [
    'customer_unique_id', # Mantener por ahora como índice

    # Features RFM (del Notebook 02)
    'recency',
    'frequency',
    'monetary',

    # Features Nuevas (del Notebook 03)
    'tenure_days',
    'avg_payment_value',
    'avg_items_per_order',
    'avg_freight_value',
    'n_unique_categories',
    'main_category',       # Dejamos esta feature categórica para el Notebook 04

    # Variable Objetivo
    'churn'
]

df_model_input = df_clientes[features_finales].copy()

# --- 3. Establecer el Índice ---
# Es una buena práctica tener el ID como índice, no como columna
df_model_input = df_model_input.set_index('customer_unique_id')

print("\n--- Dataset Final Listo para el Modelo ---")
display(df_model_input.head())

# --- 4. Guardar el Dataset Final ---
print("\nGuardando dataset final...")
df_model_input.to_parquet(DATA_DIR + "model_input.parquet")
print("✅ ¡'model_input.parquet' guardado")


Valores Nulos (NaN) antes de la limpieza:
customer_unique_id        0
last_purchase_date        0
frequency                 0
monetary                  0
recency                   0
churn                     0
first_purchase_date       0
tenure_days               0
avg_payment_value         1
avg_items_per_order       0
avg_freight_value         0
n_unique_categories       0
main_category          1303
dtype: int64

--- Dataset Final Listo para el Modelo ---


,recency,frequency,monetary,tenure_days,avg_payment_value,avg_items_per_order,avg_freight_value,n_unique_categories,main_category,churn
customer_unique_id,,,,,,,,,,
0000366f3b9a7992bf8c76cfdf3221e2,112,1,141.90,112,141.90,1.0,12.00,1,bed_bath_table,0
0000b849f77a49e4a4ce2b2a4ca5be3f,115,1,27.19,115,27.19,1.0,8.29,1,health_beauty,0
0000f46a3911fa3c0805444483337064,537,1,86.22,537,86.22,1.0,17.22,1,stationery,1
0000f6ccb0745a6a4b88665a16c9f078,321,1,43.62,321,43.62,1.0,17.63,1,telephony,1
0004aac84e0df4da2b147fca70cf8255,288,1,196.89,288,196.89,1.0,16.89,1,telephony,1



Guardando dataset final...
✅ ¡'model_input.parquet' guardado
